In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import gc
import os
import numpy as np
import shutil
import random
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix, accuracy_score
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
# Setting CUDA devices as visible
cuda_devices = "0,1"
# os.environ["CUDA_VISIBLE_DEVICES"] = cuda_devices

In [3]:
# Check if CUDA is available
if torch.cuda.is_available():
    print("Using GPUs:")
    for device_id in cuda_devices.split(','):
        device = torch.device(f"cuda:{device_id}")
        print(f"  Device {device_id}: {torch.cuda.get_device_name(int(device_id))}")
else:
    print("Using CPU")

Using GPUs:
  Device 0: Tesla P100-PCIE-16GB
  Device 1: Tesla P100-PCIE-16GB


In [4]:
torch.cuda.empty_cache()
gc.collect()

11

In [5]:
def get_classes(directory):
    class_list = []
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        if os.path.isdir(item_path) and os.listdir(item_path):
            class_list.append(item)
    return class_list

In [6]:
in_dir = "/mnt/nis_lab_research/data/class_data/neg_se/far_shah_b1-b5_b8_train_neg_se"
out_dir = "../../data/classifier/far_shah_b1-b5_b8_train_neg_se"
num_workers = os.cpu_count() or 32
epochs = 25
class_list = sorted(get_classes(in_dir))
num_classes = len(class_list)

In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to the input size expected by ResNet
    transforms.ToTensor(),
    # CHANGE TO BE DATA SPECIFIC
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [8]:
def tt_split(input_dir, out_dir, train_ratio):
    """
    Splits the dataset in the given directory into train and test sets.

    :param input_dir: Path to the input directory.
    :param train_ratio: Ratio of train set (between 0 and 1).
    """
    if not 0 <= train_ratio <= 1:
        raise ValueError("Train ratio must be between 0 and 1")

    base_dir = out_dir
    train_dir = os.path.join(base_dir, 'train')
    test_dir = os.path.join(base_dir, 'test')

    # Create train and test directories
    for directory in [train_dir, test_dir]:
        os.makedirs(directory, exist_ok=True)

    # Process each class directory
    for class_name in os.listdir(input_dir):
        class_dir = os.path.join(input_dir, class_name)
        if os.path.isdir(class_dir):
            # Create class directories in train and test
            os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
            os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

            # Get a list of images and shuffle them
            images = os.listdir(class_dir)
            random.shuffle(images)

            # Split images into train and test
            split_point = int(len(images) * train_ratio)
            train_images = images[:split_point]
            test_images = images[split_point:]

            # Copy images to train and test directories
            for image in train_images:
                shutil.copy2(os.path.join(class_dir, image), os.path.join(train_dir, class_name))
            for image in test_images:
                shutil.copy2(os.path.join(class_dir, image), os.path.join(test_dir, class_name))

In [9]:
if not os.path.exists(out_dir):
    tt_split(in_dir, out_dir, 0.8)

In [10]:
train_set = torchvision.datasets.ImageFolder(root='../../data/classifier/' + in_dir.split("/")[-1] + '/train', transform=transform)
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=num_workers)


FileNotFoundError: [Errno 2] No such file or directory: '../../data/classifier/far_shah_b1-b5_b8_train_neg_se/train'

In [ ]:
test_set = torchvision.datasets.ImageFolder(root='../../data/classifier/' + in_dir.split("/")[-1] + '/test', transform=transform)
test_loader = DataLoader(test_set, batch_size=256, shuffle=True, num_workers=num_workers)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # 27 total classes - text captcha have 0 so it is removed for now
model = model.to(device)
model = nn.DataParallel(model)  # Use DataParallel

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
for epoch in range(epochs):  # loop over the dataset multiple times
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}/{epochs}")
    for i, data in progress_bar:
        inputs, labels = data[0].to(device), data[1].to(device)  # Move data to the primary device
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        progress_bar.set_postfix(loss=running_loss / (i + 1))
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")
print('Finished Training')

In [ ]:
# Can be saved directly from the GPU
torch.save(model, './pth/test_ep25.pth')

In [ ]:
# # To load the model later
# model = torch.load('./pth/far_shah_b1-b3_rn50_ep25.pth')
# model.eval()  # Set it to evaluation mode

In [ ]:
model.eval()  # Set the model to evaluation mode

# Variables to hold predictions and actual labels
y_pred = []
y_true = []
y_score = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)  # Assuming outputs are raw scores from your model
        _, predicted = torch.max(outputs.data, 1)
        
        # Accumulate true labels and predictions
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())
        y_score.extend(probabilities.cpu().numpy())
        

In [ ]:
len(set(y_true))

In [ ]:
gt_by_class = []
for i in range (0, num_classes):
    tmp = []
    for j, gt in enumerate(y_true):
        if i == gt:
            tmp.append([gt, y_pred[j]])
        
    gt_by_class.append(tmp)
     

In [ ]:
tot = 0 
for i, class_ in enumerate(gt_by_class):
    class_cntr = 1
    for inst in class_:
        if inst[0] == inst[1]:
            class_cntr += 1
    print(i, class_list[i], class_cntr)
    tot = tot + class_cntr / len(class_)

In [ ]:
tot/len(gt_by_class)

In [ ]:
# Ensure the directory exists
os.makedirs('res', exist_ok=True)

In [ ]:

# Open a file to write the outputs
with open('res/results.txt', 'w') as f:

    # Convert accumulated predictions and labels to numpy arrays
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    y_score = np.array(y_score)

    # Determine the unique classes in y_true and binarize
    classes = np.unique(y_true)  # Identify unique class labels
    y_true_binarized = label_binarize(y_true, classes=classes)

    n_classes = len(classes)

    # Calculate metrics
    accuracy = np.mean(y_pred == y_true)
    precision = precision_score(y_true, y_pred, average="weighted", labels=classes)
    recall = recall_score(y_true, y_pred, average="weighted", labels=classes)
    f1 = f1_score(y_true, y_pred, average="weighted", labels=classes)

    # ROC Curve and AUC for Micro-average
    fpr, tpr, _ = roc_curve(y_true_binarized.ravel(), y_score.ravel())
    roc_auc = auc(fpr, tpr)

    # Print and write metrics to file
    metrics_output = f"OVERALL METRICS\nAccuracy: {accuracy * 100:.2f}%\nPrecision: {precision:.4f}\nRecall: {recall:.4f}\nF1 Score: {f1:.4f}\n"
    print(metrics_output)
    f.write(metrics_output)

    # Calculate accuracy by class
    accuracies_by_class = {}
    for cls in classes:
        idx = np.where(y_true == cls)[0]
        accuracy_cls = accuracy_score(y_true[idx], y_pred[idx])
        accuracies_by_class[cls] = accuracy_cls

    # Print and write accuracy by class to file
    f.write("\nACCURACY BY CLASS\n")
    for cls, acc in accuracies_by_class.items():
        class_output = f'{class_list[cls]} (class {cls}): {acc * 100:.2f}%\n'
        print(class_output)
        f.write(class_output)

    # Calculate and visualize the confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=classes)
    plt.figure(figsize=(20, 12))
    sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.savefig('res/confusion_matrix.png')
    plt.show()

    # Plot ROC Curve for Micro-average
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange', lw=lw, label='Micro-average ROC curve (area = {0:0.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic - Micro-average')
    plt.legend(loc='lower right')
    plt.show()